In [18]:
import csv,re
from datetime import datetime
import spacy
import random

In [2]:
T1 = "Who was the {} between {} and {} at {}?"
T2 = "Who was the {} between {} and {} on {}?"
T3 = "Where was the match between {} and {} held on {}?"
T4 = "When was the match between {} and {} held at {}?"
T5 = "How many {} were scored by {} in the match between {} and {} at {}?"
T6 = "How many {} were scored by {} in the match between {} and {} on {}?"
T7 = "How many {} were scored against {} in the match between {} and {} at {}?"
T8 = "How many {} were scored against {} in the match between {} and {} on {}?"
T7B = "How many {} did {} bowl in the match between {} and {} at {}?"
T8B = "How many {} did {} bowl in the match between {} and {} on {}?"

batCol = ['runs', 'fours', 'sixes']
bowlCol = ['overs', 'no balls', 'wide balls']
quesList = []
quesDict = {}
quesDict["1"] = []
quesDict["2"] = []
quesDict["3"] = []
quesDict["4"] = []
quesDict["5"] = []
quesDict["6"] = []
quesDict["7A"] = []
quesDict["7B"] = []
quesDict["8A"] = []
quesDict["8B"] = []

In [3]:
with open('../data/match-data.csv','r') as f:
    table = csv.reader(f, delimiter=',')
    table = list(table)[1:]
    for i in range(len(table)):
        row = table[i]
        team1 = row[0]
        team2 = row[1]
        ground = row[4]
        date = row[5]
        date = datetime.strptime(date,"%b %d, %Y").strftime("%dDate/%m/%Y")
        quesList.append(T1.format('winner', team1, team2, ground))
        quesList.append(T1.format('loser', team1, team2, ground))
        quesList.append(T2.format('winner', team1, team2, date))
        quesList.append(T2.format('loser', team1, team2, date))
        quesList.append(T3.format(team1, team2, date))
        quesList.append(T4.format(team1, team2, ground))
        quesDict["1"].append(T1.format('winner', team1, team2, ground))
        quesDict["1"].append(T1.format('loser', team1, team2, ground))
        quesDict["2"].append(T2.format('winner', team1, team2, date))
        quesDict["2"].append(T2.format('loser', team1, team2, date))
        quesDict["3"].append(T3.format(team1, team2, date))
        quesDict["4"].append(T4.format(team1, team2, ground))
        
        
        with open('../data/scoreboard-match-'+str(i+1)+'-batting.csv','r') as sc:
            board = csv.reader(sc, delimiter= ',')
            board = list(board)
            for j in range(len(board)):
                srow = board[j]
                if len(srow) == 0 or srow[0] == "BATSMEN":
                    continue
                batsman = srow[0]
                batsman = (re.split("[^a-zA-Z\s]", batsman)[0]).strip()
                for col in batCol:
                    quesList.append(T5.format(col,batsman,team1,team2,ground))
                    quesList.append(T6.format(col,batsman,team1,team2,date))
                    quesDict["5"].append(T5.format(col,batsman,team1,team2,ground))
                    quesDict["6"].append(T6.format(col,batsman,team1,team2,date))
                    
        with open('../data/scoreboard-match-'+str(i+1)+'-bowling.csv','r') as sc:
            board = csv.reader(sc, delimiter= ',')
            board = list(board)
            for j in range(len(board)):
                srow = board[j]
                if len(srow) == 0 or srow[0] == "Bowling":
                    continue
                bowler = srow[0]
                for col in batCol:
                    quesList.append(T7.format(col,bowler,team1,team2,ground))
                    quesList.append(T8.format(col,bowler,team1,team2,date))
                    quesDict["7A"].append(T7.format(col,bowler,team1,team2,ground))
                    quesDict["8A"].append(T8.format(col,bowler,team1,team2,date))
                    
                for col in bowlCol:
                    quesDict["7B"].append(T7B.format(col,bowler,team1,team2,ground))
                    quesDict["8B"].append(T8B.format(col,bowler,team1,team2,date))

In [4]:
len(quesList)

10254

In [5]:
print(quesDict["1"][0])
print(quesDict["2"][0])
print(quesDict["3"][0])
print(quesDict["4"][0])

Who was the winner between Mum Indians and Super Kings at Mumbai?
Who was the winner between Mum Indians and Super Kings on 07Date/04/2018?
Where was the match between Mum Indians and Super Kings held on 07Date/04/2018?
When was the match between Mum Indians and Super Kings held at Mumbai?


In [6]:
train_data = []
for sen in quesDict["1"]:
    tup = (sen, {"entities": [(sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("at") - 1, "Team"),
                              (sen.find("at") + 3, sen.find("?"), "Ground")]})
    train_data.append(tup)

for sen in quesDict["2"]:
    tup = (sen, {"entities": [(sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("on") - 1, "Team"),
                              (sen.find("on") + 3, sen.find("?"), "Date")]})
    train_data.append(tup)
    
for sen in quesDict["3"]:
    tup = (sen, {"entities": [(sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("held") - 1, "Team"),
                              (sen.find("on") + 3, sen.find("?"), "Date")]})
    train_data.append(tup)
    
for sen in quesDict["4"]:
    tup = (sen, {"entities": [(sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("held") - 1, "Team"),
                              (sen.find("at") + 3, sen.find("?"), "Ground")]})
    train_data.append(tup)
   

In [7]:
print(quesDict["5"][0])
print(quesDict["6"][0])

How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?
How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings on 07Date/04/2018?


In [8]:
for sen in quesDict["5"]:
    tup = (sen, {"entities": [(sen.find("by") + 3, sen.find("in") - 1, "Player"),
                              (sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("on") - 1, "Team"),
                              (sen.find("at") + 3, sen.find("?"), "Ground")]})
    train_data.append(tup)

for sen in quesDict["6"]:
    tup = (sen, {"entities": [(sen.find("by") + 3, sen.find("in") - 1, "Player"),
                              (sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("held") - 1, "Team"),
                              (sen.find("on") + 3, sen.find("?"), "Date")]})
    train_data.append(tup)


In [9]:
print(quesDict["7A"][0])
print(quesDict["8A"][0])
print(quesDict["7B"][0])
print(quesDict["8B"][0])

How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?
How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings on 07Date/04/2018?
How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?
How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings on 07Date/04/2018?


In [10]:
for sen in quesDict["7A"]:
    tup = (sen, {"entities": [(sen.find("against") + 8, sen.find("in") - 1, "Player"),
                              (sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("at") - 1, "Team"),
                              (sen.find("at") + 3, sen.find("?"), "Ground")]})
    train_data.append(tup)
    
for sen in quesDict["8A"]:
    tup = (sen, {"entities": [(sen.find("against") + 8, sen.find("in") - 1, "Player"),
                              (sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("on") - 1, "Team"),
                              (sen.find("on") + 3, sen.find("?"), "Date")]})
    train_data.append(tup)
    
for sen in quesDict["7B"]:
    tup = (sen, {"entities": [(sen.find("did") + 4, sen.find("bowl") - 1, "Player"),
                              (sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("at") - 1, "Team"),
                              (sen.find("at") + 3, sen.find("?"), "Ground")]})
    train_data.append(tup)

for sen in quesDict["8A"]:
    tup = (sen, {"entities": [(sen.find("did") + 4, sen.find("bowl") - 1, "Player"),
                              (sen.find("between") + 8, sen.find("and") - 1, "Team"), 
                              (sen.find("and") + 4, sen.find("on") - 1, "Team"),
                              (sen.find("on") + 3, sen.find("?"), "Date")]})
    train_data.append(tup)

In [14]:
print(len(train_data))

14550


In [15]:
quesList

['Who was the winner between Mum Indians and Super Kings at Mumbai?',
 'Who was the loser between Mum Indians and Super Kings at Mumbai?',
 'Who was the winner between Mum Indians and Super Kings on 07Date/04/2018?',
 'Who was the loser between Mum Indians and Super Kings on 07Date/04/2018?',
 'Where was the match between Mum Indians and Super Kings held on 07Date/04/2018?',
 'When was the match between Mum Indians and Super Kings held at Mumbai?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings on 07Date/04/2018?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings on 07Date/04/2018?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many si

In [6]:
with open('../data/queslist.txt', 'w') as f:
    for x in quesList:
        f.write(str(x) + "\n")

In [19]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(train_data, 20)

# Save our trained Model
# modelfile = input("Enter your Model Name: ")
prdnlp.to_disk("ner")

#Test your text
# test_text = 
doc = prdnlp("How many runs were scored by t k temp in the match between Ind and Ind at Mumbai stadium?")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Statring iteration 0
{'ner': 10766.475042393264}
Statring iteration 1
{'ner': 9444.767989862901}
Statring iteration 2
{'ner': 8852.080229240013}
Statring iteration 3
{'ner': 8431.998363356019}
Statring iteration 4
{'ner': 8342.553423370497}
Statring iteration 5
{'ner': 8516.996549312904}
Statring iteration 6
{'ner': 8199.114120671447}
Statring iteration 7
{'ner': 8412.338816454256}
Statring iteration 8
{'ner': 9454.447336825428}
Statring iteration 9
{'ner': 10240.35406749156}
Statring iteration 10
{'ner': 10858.795545602401}
Statring iteration 11
{'ner': 10513.932664191503}
Statring iteration 12
{'ner': 10507.734848954942}
Statring iteration 13
{'ner': 10949.156811713425}
Statring iteration 14
{'ner': 11002.134888713254}
Statring iteration 15
{'ner': 11378.807499308734}
Statring iteration 16
{'ner': 11222.312548090867}
Statring iteration 17
{'ner': 11605.75093704078}
Statring iteration 18
{'ner': 11992.523906671942}
Statring iteration 19
{'ner': 11366.424361899863}
t k temp 29 37 Playe

In [23]:
doc = prdnlp("How many runs were scored by t k temp in the match between Ind and temp at Mumbai stadium?")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

t k temp 29 37 Player
Ind 59 62 Team
stadium 82 89 Date
